In [9]:
import pandas as pd
data=pd.read_csv('C:/Users/Ajit.Dange/Desktop/LSG Dashboard/Lsg Dashboard jan to april20.csv',low_memory=False,nrows = 4000000, error_bad_lines = False)


import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import norm
import statistics
data.dropna(subset=['Target_Date'],inplace=True)
data.dropna(subset=['Disposal_Date'],inplace=True)
data.dropna(subset=['Target_Date'],inplace=True)
data.drop(data.index[data['Samadhan_One'] == 'Y'],inplace=True)
data['EP_Trans_Date'] = pd.to_datetime(data['EP_Trans_Date'])
data['Target_Date'] = pd.to_datetime(data['Target_Date'])
data['DtOfAppCompletion'] = pd.to_datetime(data['DtOfAppCompletion'])
data['Lsk_Fwd_Date'] = pd.to_datetime(data['Lsk_Fwd_Date'])
data['Reg Date'] = pd.to_datetime(data['Reg Date'])
data['Disposal_Date']=pd.to_datetime(data['Disposal_Date'], errors = 'coerce')
data['appprocesstime'] = data['Disposal_Date'] - data['Reg Date']
data['appprocesstime']=data['appprocesstime']/np.timedelta64(1,'D')
data['dvt'] = data['Disposal_Date'] - data['Target_Date']
data['dvt']=data['dvt']/np.timedelta64(1,'D')



data.reset_index(inplace = True, drop = True)



Q1 = np.percentile(data['dvt'], 25) 
Q3 = np.percentile(data['dvt'], 75)                  
IQR = Q3 - Q1 


upper = np.where(data['dvt'] >= (Q3+8*IQR))

lower = np.where(data['dvt'] <= (Q1-8*IQR))

data.drop(upper[0], inplace = True)

data.drop(lower[0], inplace = True)

from ipywidgets import widgets, interactive
from io import StringIO
district = widgets.Dropdown(
    options=['All'] + list(data['District_Name_E'].unique()),
    value='All',
    description='Districts:')
service = widgets.Dropdown(
    options=['All'] + list(data['Service_Name_E'].unique()),
    value='All',
    description='Services:')



def plotit(district,service):
    if district== 'All' and service == 'All':
        rslt_df=data
    elif district== 'All' and service == service:
        rslt_df = data.loc[data["Service_Name_E"] == service]
    elif district== district and service == 'All':
        rslt_df = data.loc[data["District_Name_E"] == district]
        
    else:
        rslt_df = data.loc[(data["District_Name_E"] == district) & (data["Service_Name_E"] == service)]
        
    
    
    fig, ax = plt.subplots(figsize=(15, 8))

    sns.distplot(rslt_df['dvt'], hist=True, rug=True,kde=True,ax=ax)
    ax.set(xlabel='Disposal days', ylabel='Probability function value', title=f'Distribution curve for: {service}')
    ax.set_xlim(-10,5)
    ax.set_xticks(range(-10,5))
    mean=rslt_df['dvt'].mean()
    std=rslt_df['dvt'].std()
    p=norm.cdf(1, mean, std)
    p1=int((1-p)*100)
    a1=int(rslt_df['dvt'].count())
    m1=int(min(rslt_df['dvt'],default = "EMPTY"))
    m2=int(rslt_df['dvt'].max())
    m3=int(rslt_df['dvt'].mean())
    ax.text(2,2,f' Total Applications : {a1}\n Minimum Disposal time : {m1} days\n Maximum Disposal time : {m2} days\n Mean Disposal time : {m3} days\n %  of applications Exceeding timeline : {p1} %',size=18,
          ha="right", multialignment="left",style='italic',
         bbox={'facecolor': 'green', 'alpha': 0.5, 'pad': 10})
    ax.plot([2], [1], 'o')
    fig = plt.figure(figsize = (18,8))
    plt.show()
    


interactive(plotit, district=district,service=service)


interactive(children=(Dropdown(description='Districts:', options=('All', 'Agar Malwa', 'Khandwa', 'Burhanpur',…